# wat is de connectie tussen vertragingen en het weer.

dit noteboek is bedoeld voor AI4 semester Fontys ICT, eigen chalange. 

in dit notebook ga ik proberen te achterhallen of het weer een roll speelt met een vertraging in het openbaren trein verkeer.

met deze datasets hoop ik te kunnen vinden of en waar er een goede verbinding is.


in mijn chalage ga ik gebruik maken van phyton pandas, een library om datasets in te laden. ik ga daarnaast ook gebruik maken van plotly om het mooi aan te geven en sklearn voor de machine learning algoritmes.

dependencies voor de chalange:

In [112]:
# dependacies
import os
import numpy as np
# import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
# from geopy.exc import GeocoderTimedOut
# from geopy.geocoders import Nominatim
# from geopy.distance import geodesic
import plotly.express as px
import plotly.graph_objects as go

from sklearn.impute import KNNImputer

datafolder = 'data/'

max_data = datetime(2023, 3, 31, 23, 59, 59)

# 📦 Data opzet

nu ben ik op zoek gegaan naar datasets die van belang kunnen zijn in mijn opdracht. 

dit zijn data sets die ik heb gekozen:
- [rijden de trijnen. Treinarchief](https://www.rijdendetreinen.nl/open-data/treinarchief)
- [rijden de trijnen. treinstoringen](https://www.rijdendetreinen.nl/open-data/treinstoringen)
- [KNMI weerstations Uurwaarnemingen](https://www.daggegevens.knmi.nl/klimatologie/uurgegevens)
- [Train Stations in Europe](https://www.kaggle.com/datasets/headsortails/train-stations-in-europe)


in deze datasets staat het all het trein verkeer, het trein verkeer dat van gezecht word dat het een storing was, het weer in uren en de locatie van het trein station.

### 🚄 treinarchief laden.

de code hieronder laad het trainarchief in. in dit treinarchief hebben we de volgende colomen volgens [rijden de trijnen](https://www.rijdendetreinen.nl/):
- `Service:RDT-ID` Dit ID wordt gebruikt voor deze rit. Het heeft geen nuttige betekenis buiten het uniek identificeren van een enkele rit op een enkele datum.
>Dit ID zal meerdere keren voorkomen in de CSV-bestanden omdat dit ID uniek is voor iedere rit. Er is ook een kolom Stop:RDT-ID met een uniek ID voor iedere stop.
- `Service:Date` De geplande datum voor deze rit.
>De ritdatum is niet altijd de logische datum. Bijvoorbeeld: een rit die vertrekt om 23:59 op 31 juli en aankomt om 02:00 op 1 augustus heeft een ritdatum van 31 juli. Vertragingen hebben geen invloed op de ritdatum.
- `Service:Type` Het soort trein, zoals Intercity, Sprinter of ICE International.
- `Service:Company` Vervoerder die deze rit uitvoert, zoals NS of Arriva.
- `Service:Train number` Het treinnummer (ritnummer) voor deze rit identificeert deze rit uniek op deze datum. Dit nummer wordt soms ook gecommuniceerd naar reizigers (vooral voor internationale treinen). 
> Een enkele rit kan soms meerdere treinnummers hebben. Bijvoorbeeld wanneer een trein gesplitst wordt in twee delen, of wanneer een trein op een groot station halverwege van treinnummer verandert.
- `Service:Completely cancelled` Deze kolom is true wanneer alle stops van deze rit opgeheven zijn. Of met andere woorden: wanneer de trein helemaal niet rijdt.
- `Service:Partly cancelled` Deze kolom is true wanneer een of meer stops van deze rit opgeheven zijn. Of met andere woorden: wanneer de trein niet rijdt op een deel van het traject.
- `Service:Maximum delay` De hoogste vertraging (in minuten) van alle stops van deze rit.
- `Stop:RDT-ID` Unieke ID voor deze stop. Dit ID is uniek voor iedere stop in de dataset. Het heeft verder geen nuttige betekenis.
- `Stop:Station code` Code (verkorting) van de stationsnaam.
- `Stop:Station name` De naam van het station.
- `Stop:Arrival time` Geplande aankomsttijd in RFC 3339-formaat. Deze kolom is leeg wanneer er geen aankomst gepland was.
- `Stop:Arrival delay` Vertraging bij aankomst in minuten. Deze kolom is leeg wanneer er geen aankomst gepland was.
- `Stop:Arrival cancelled` Deze kolom is true wanneer de aankomst op deze stop opgeheven is. Deze kolom is leeg wanneer er geen aankomst gepland was.
- `Stop:Departure time` Geplande vertrektijd in RFC 3339-formaat. Deze kolom is leeg wanneer er geen vertrek gepland was.
- `Stop:Departure delay` Vertraging bij vertrek in minuten. Deze kolom is leeg wanneer er geen vertrek gepland was.
- `Stop:Departure cancelled` Deze kolom is true wanneer het vertrek op deze stop opgeheven is. Deze kolom is leeg wanneer er geen vertrek gepland was.


In de code laad ik de dataset in en verander ik de datum naar een datetime zodat ik hem dan kan verminderen naar het zoekgebied die ik op het moment heb in de weer dataset. daarna zet ik type en bedrijf naar kleine letters omdat de dataset handmatig soms is aangepast en dan worden hoofdletter vergeten.

In [113]:
Train_trafic_2023 = pd.read_csv(datafolder + 'services-2023/services-2023.csv', sep=',', low_memory=False)

Train_trafic_2023['Service:Date'] = pd.to_datetime(Train_trafic_2023['Service:Date'], format="%Y-%m-%d")

Train_trafic_2023['Service:Type'] = Train_trafic_2023['Service:Type'].str.lower()
Train_trafic_2023['Service:Company'] = Train_trafic_2023['Service:Company'].str.lower()
Train_trafic_2023['Stop:Station name'] = Train_trafic_2023['Stop:Station name'].str.lower()

# Train_trafic_2023 = Train_trafic_2023[Train_trafic_2023['Service:Company'] == 'ns']

Train_trafic_2023 = Train_trafic_2023[Train_trafic_2023['Service:Date'] < max_data]

print(f'er zijn {len(Train_trafic_2023)} entries ingeladen') 

er zijn 5142250 entries ingeladen


### 🌥️ KNMI Uurwaarnemingen laden

in de code hieronder laad ik de uurwaarneming van het weer in. dit is waarom meerendeels van het project alleen over de eerse 2 maanden van het project gaan. <p>
in de dataset van het weer hebben we:
- `STN` dit is het weer station nummer
- `YYYYMMDD` datum (YYYY=jaar;MM=maand;DD=dag)
- `start_time` tijd (HH=uur; UT.12 UT=13 MET; 14 MEZT. Uurvak 05 loopt van 04.00 UT tot 5.00 UT)
- `DD` Windrichting (in graden) gemiddeld over de laatste 10 minuten van het afgelopen uur (360=noord; 90=oost; 180=zuid; 270=west; 0=windstil 990=veranderlijk.
- `FH` Uurgemiddelde windsnelheid (in 0.1 m/s).
- `FF` Windsnelheid (in 0.1 m/s) gemiddeld over de laatste 10 minuten van het afgelopen uur.
- `FX` Hoogste windstoot (in 0.1 m/s) over het afgelopen uurvak.
- `temp` Temperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming.
- `T10N` Minimumtemperatuur (in 0.1 graden Celsius) op 10 cm hoogte in de afgelopen 6 uur.
- `TD` Dauwpuntstemperatuur (in 0.1 graden Celsius) op 1.50 m hoogte tijdens de waarneming.
- `SQ` Duur van de zonneschijn (in 0.1 uren) per uurvak; berekend uit globale straling  (-1 for <0.05 uur).
- `Q` Globale straling (in J/cm2) per uurvak.
- `DR` Duur van de neerslag (in 0.1 uur) per uurvak.
- `RH` Uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm).
- `P` Luchtdruk (in 0.1 hPa) herleid naar zeeniveau; tijdens de waarneming.
- `VV` Horizontaal zicht tijdens de waarneming (0=minder dan 100m; 1=100-200m; 2=200-300m;...; 49=4900-5000m; 50=5-6km; 56=6-7km; 57=7-8km; ...; 79=29-30km; 80=30-35km; 81=35-40km;...; 89=meer dan 70km).
- `N` Bewolking (bedekkingsgraad van de bovenlucht in achtsten); tijdens de waarneming (9=bovenlucht onzichtbaar).
- `U` Relatieve vochtigheid (in procenten) op 1.50 m hoogte tijdens de waarneming.
- `WW` Weercode (00-99); visueel(WW) of automatisch(WaWa) waargenomen; voor het actuele weer of het weer in het afgelopen uur.
- `IX` Weercode indicator voor de wijze van waarnemen op een bemand of automatisch station (1=bemand gebruikmakend van code uit visuele waarnemingen; 2;3=bemand en weggelaten (geen belangrijk weersverschijnsel; geen gegevens); 4=automatisch en opgenomen (gebruikmakend van code uit visuele waarnemingen); 5;6=automatisch en weggelaten (geen belangrijk weersverschijnsel; geen gegevens); 7=automatisch gebruikmakend van code uit automatische waarnemingen).
- `M` Mist 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming.
- `R` Regen 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming.
- `S` Sneeuw 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming.
- `O` Onweer 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming.
- `Y` IJsvorming 0=niet voorgekomen; 1=wel voorgekomen in het voorgaande uur en/of tijdens de waarneming.


hier laad ik het weer in dat en zet de start_time of de start tijd naar een datetime. daarby combineer ik twee colommen om een datum en tijd in 1 colom te krijgen.

In [114]:
weather_2023 = pd.read_csv(datafolder + '2023-jan-mar.csv', sep=',', low_memory=False, encoding='utf-8', skiprows=80)

weather_2023['start_time'] = weather_2023['start_time'].astype(str) + ' ' + (weather_2023['HH'] - 1).astype(str) + ':00:00'

weather_2023['start_time'] = pd.to_datetime(weather_2023['start_time'], format="%Y%m%d %H:%M:%S")
#  drop columns that are not needed
# weather_2023 = weather_2023.drop(columns=['HH', 'WW', 'IX', 'FH', 'FX', 'TD', 'SQ', 'RH'])
weather_2023 = weather_2023.drop(columns=['HH'])

print(f'er zijn {len(weather_2023)} entries ingeladen') 

er zijn 99360 entries ingeladen


de weer dataset bestaat uit 2 datasets. de gene hierboven die voor de gegevens uit de weer stations.
en de gene hieronder die gaat over de weer station Locaties.


de weer station locatie dataset heeft de collomen:
- `STN` dit is het weer station nummer.
- `LON(east)` dit is de longitude van het weer station.
- `LAT(north)` dit is de latitude van het weer station.
- `ALT(m)` dit is de hoogte waarop het station staat.
- `NAME` de naam van waar het weer station staat.

met deze gegevens hoop ik te achterhalen wat het weer was het station van aankoms.

In [115]:
weather_station_location = pd.read_csv(datafolder + 'Weather-stations-loc.csv', sep=',', low_memory=False, encoding='utf-8')
print(f'er zijn {len(weather_station_location)} entries ingeladen') 

er zijn 50 entries ingeladen


### 🏟️ trein station 

in de dataset hieronder zitten alle trein stations van europa. ik zal minder nodig hebben dus zal met de country alleen nederland, duitserland en belgie gaan houden. dit is ook omdat mijn andere datasets niet verder kijken dan die locaties.

in deze dataset zit:
- `id` de primary sleutel van de data.
- `name` Naam van het station zoals het lokaal bekend is. Deze namen bevatten accenten en andere speciale tekens.
- `name_norm` De gelatiniseerde versie van de naam.
- `uic` International Union of Railways (UIC) code.
- `latitude` Coordinates
- `longitude` Coordinates
- `parent_station_id` A station can belong to a meta station whose `id` is this value.
- `country` Landcodes in ISO 3166-1 alpha-2 formaat (2 cijfers).
- `time_zone` Continent/Land ISO-codes. Deze lijken equivalent te zijn aan Olson-namen (bijvoorbeeld “Europe/Berlin”).
- `is_city` Gemarkeerd als “onbetrouwbaar” in de bron dataset.
- `is_main_station` 
- `is_airport` 
- `entur_id` 
- `entur_is_enabled` 

ik zal deze dataset gebruiken om de locaties van de trein stations te vinden. <p>
en aangezien ik alleen de locaties hoef te hebben zal ik die ook allen onthouden. naast de belangerijke data zoals Namen die het kunnen zijn en in welk land ze zitten.

met de hulp van deze gegevens hoop ik de gaten in mijn gegevens te vullen. 


In [116]:
europa_train_station_locations = pd.read_csv(datafolder + 'train_stations_europe.csv', low_memory=False, encoding='utf-8')

europa_train_station_locations = europa_train_station_locations.drop(columns=['id', 'uic', 'parent_station_id', 'time_zone', 'is_city', 'is_main_station', 'is_airport', 'entur_id', 'entur_is_enabled'])

europa_train_station_locations['country'] = pd.Categorical(europa_train_station_locations['country'], categories=['NL', 'DE', 'BE'], ordered=True)
europa_train_station_locations = europa_train_station_locations[europa_train_station_locations['country'].notna()]

europa_train_station_locations['name'] = europa_train_station_locations['name'].str.lower()

print(f'er zijn {len(europa_train_station_locations)} entries ingeladen') 

er zijn 8934 entries ingeladen


# 📃 Sample de data

om even naar de data te kijken ga ik 20 van iedere dataset hieronder laden. dit is om te kijken of het wel goed is ingeladen en dat de data er een beetje werkzaam uitziet.

### 🚄 treinarchief laden.

hier laad ik de samples van het trein verkeer.

In [117]:
sample_size = 20

display(Train_trafic_2023.sample(sample_size))

,Service:RDT-ID,Service:Date,Service:Type,Service:Company,Service:Train number,Service:Completely cancelled,Service:Partly cancelled,Service:Maximum delay,Stop:RDT-ID,Stop:Station code,Stop:Station name,Stop:Arrival time,Stop:Arrival delay,Stop:Arrival cancelled,Stop:Departure time,Stop:Departure delay,Stop:Departure cancelled
4263455,10823586,2023-03-17,intercity,ns,1714,False,False,0,97336246,UT,utrecht centraal,2023-03-17T06:23:00+01:00,1.0,False,2023-03-17T06:28:00+01:00,0.0,False
1056463,10476866,2023-01-20,sprinter,ns,3327,False,False,0,94129256,HN,hoorn,2023-01-20T07:18:00+01:00,0.0,False,2023-01-20T07:20:00+01:00,0.0,False
4739770,10872296,2023-03-25,stoptrein,arriva,32031,False,False,0,97812432,LG,landgraaf,2023-03-25T11:44:00+01:00,0.0,False,2023-03-25T11:44:00+01:00,0.0,False
4365319,10833771,2023-03-18,intercity,ns,3678,False,False,0,97438081,ZP,zutphen,2023-03-18T23:02:00+01:00,0.0,False,2023-03-18T23:03:00+01:00,0.0,False
2699751,10652494,2023-02-17,sprinter,ns,4865,False,False,0,95772569,HWD,heerhugowaard,2023-02-17T18:09:00+01:00,1.0,False,2023-02-17T18:09:00+01:00,1.0,False
1096529,10480862,2023-01-20,sprinter,r-net,8662,False,False,0,94169325,APN,alphen a/d rijn,2023-01-20T19:20:00+01:00,0.0,False,NaN,NaN,NaN
2108229,10590166,2023-02-07,sprinter,ns,4053,False,False,0,95181043,KZ,koog aan de zaan,2023-02-07T14:02:00+01:00,1.0,False,2023-02-07T14:02:00+01:00,1.0,False
2523324,10634486,2023-02-14,stoptrein,valleilijn,31352,False,False,0,95596139,HVL,hoevelaken,2023-02-14T18:54:00+01:00,0.0,False,2023-02-14T18:55:00+01:00,0.0,False
430631,10411910,2023-01-09,sprinter,ns,6437,False,True,0,93503414,BTL,boxtel,2023-01-09T11:51:00+01:00,0.0,False,2023-01-09T11:52:00+01:00,0.0,False
3842197,10779154,2023-03-09,stoptrein,db,20273,False,False,0,96915012,ESEM,selm,2023-03-09T17:35:00+01:00,0.0,False,2023-03-09T17:35:00+01:00,0.0,False


### 🌥️ KNMI Uurwaarnemingen laden
hieronder laad ik de samples can de weerstations.

In [118]:
display(weather_2023.sample(sample_size))
display(weather_station_location.sample(sample_size))

,STN,start_time,DD,FH,FF,FX,temp,T10N,TD,SQ,...,VV,N,U,WW,IX,M,R,S,O,Y
49729,286,2023-01-03 01:00:00,210.0,40.0,40.0,50.0,32.0,NaN,23.0,0.0,...,NaN,NaN,93.0,NaN,6,NaN,NaN,NaN,NaN,NaN
71613,324,2023-01-14 21:00:00,250.0,90.0,110.0,140.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
40390,278,2023-03-04 22:00:00,270.0,20.0,20.0,40.0,31.0,NaN,17.0,0.0,...,NaN,NaN,90.0,NaN,6,NaN,NaN,NaN,NaN,NaN
8624,235,2023-03-31 08:00:00,190.0,80.0,90.0,140.0,115.0,NaN,97.0,0.0,...,70.0,8.0,88.0,23.0,7,0.0,1.0,0.0,0.0,0.0
59703,312,2023-02-27 15:00:00,60.0,80.0,90.0,110.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
25926,267,2023-01-01 06:00:00,220.0,110.0,110.0,150.0,101.0,NaN,78.0,0.0,...,63.0,8.0,85.0,NaN,5,0.0,0.0,0.0,0.0,0.0
16998,249,2023-03-20 06:00:00,210.0,50.0,50.0,70.0,73.0,NaN,52.0,0.0,...,65.0,NaN,86.0,NaN,5,0.0,0.0,0.0,0.0,0.0
90794,375,2023-01-04 02:00:00,210.0,100.0,90.0,140.0,93.0,NaN,84.0,0.0,...,60.0,8.0,93.0,81.0,7,0.0,1.0,0.0,0.0,0.0
21853,258,2023-01-11 13:00:00,240.0,120.0,120.0,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN
26880,267,2023-02-10 00:00:00,220.0,40.0,40.0,60.0,33.0,NaN,27.0,0.0,...,64.0,8.0,95.0,NaN,5,0.0,0.0,0.0,0.0,0.0


,STN,LON(east),LAT(north),ALT(m),NAME
42,348,4.926,51.970,-0.7,CabauwMast
18,275,5.873,52.056,48.2,Deelen
10,257,4.603,52.506,8.5,WijkaanZee
47,377,5.763,51.198,30.0,Ell
30,312,3.622,51.768,0.0,Oosterschelde
43,350,4.936,51.566,14.9,Gilze-Rijen
3,225,4.555,52.463,4.4,IJmuiden
32,315,3.998,51.447,0.0,Hansweert
20,278,6.259,52.435,3.6,Heino
36,324,4.006,51.596,0.0,Stavenisse


### 🏟️ trein station 
hier laad ik de samples van de trein stations

In [119]:
display(europa_train_station_locations.sample(sample_size))

,name,name_norm,latitude,longitude,country
9269,beringhausen,Beringhausen,51.407771,8.751412,DE
13547,regis-breitingen,Regis-Breitingen,51.083126,12.426193,DE
5096,aachen,Aachen,50.776070,6.083780,DE
10311,hoheneggelsen,Hoheneggelsen,52.200179,10.197821,DE
13244,leutenberg,Leutenberg,50.564817,11.447861,DE
4527,gent-sint-pieters,Gent-Sint-Pieters,51.035627,3.710612,BE
26900,frankenberg-goßberg,Frankenberg-Gossberg,51.064995,8.799324,DE
5434,lahr (schwarzw),Lahr (Schwarzw),48.340436,7.834737,DE
10995,neusorg,Neusorg,49.933846,11.971986,DE
10909,münchen-neuaubing,Munchen-Neuaubing,48.141577,11.421693,DE


# 🛠️ Preprocessing

aangezien ik meerdere datasets heb moet ik als eers even kijken waar de vertragingen zitten.

hiervoor maak ik een pie chard en kijk ik wat er de vertragingen zijn.

### 🚄 trein verkeer 
in het trein verkeer is het belangerijk om te weten dat als er null staat dan heeft het nog steeds een betekenis. 

In [120]:
# Train_trafic_2023['binned'] = pd.cut(Train_trafic_2023['Stop:Departure delay'], bins=range(0, Train_trafic_2023['Stop:Departure delay'].max() + 10, 10), include_lowest=True)
bins = list(range(1, 10, 1)) + list(range(10, 70, 10)) + list(range(70, int(Train_trafic_2023['Stop:Departure delay'].max()) +100, 100))

# Define your custom labels
labels = ['0'] +  [f'{bins[i]}-{bins[i+1]}' for i in range(len(bins)-1)]

bins = [0] + bins 

Train_trafic_2023['binned vertrek vertraging'] = pd.cut(Train_trafic_2023['Stop:Departure delay'], bins=bins, labels=labels, include_lowest=True)
Train_trafic_2023['binned vertrek vertraging'] = Train_trafic_2023['binned vertrek vertraging'].astype(str)

counts = Train_trafic_2023['binned vertrek vertraging'].value_counts(sort=False)
fig = px.pie(values=counts, names=counts.index, title='vertrek vertraging in minuten.')
fig.show()

in de chard hierboven is in te lezen dat de meeste treinen op tijd weggaan van hun locatie. houd nogmaals rekening dat 0 en nan allebij zeggen dat er geen vertraging vermeld is.

en als ik de nan en 0 weg haal is de vertraging voornamelijk binnen de 10 minuten. ( je kunt zelf ook de 0 en nan weghalen in de pie chard. klick op de namen die u weg will hebben )

maar dit hierboven is alleen de vertrek vertraging. hieronder heb ik de aankomst op de zelfde manier gedaan.

In [121]:
bins = list(range(1, 10, 1)) + list(range(10, 70, 10)) + list(range(70, int(Train_trafic_2023['Stop:Arrival delay'].max()) +100, 100))

# Define your custom labels
labels = ['0'] +  [f'{bins[i]}-{bins[i+1]}' for i in range(len(bins)-1)]

bins = [0] + bins 

Train_trafic_2023['binned aankomst vertraging'] = pd.cut(Train_trafic_2023['Stop:Arrival delay'], bins=bins, labels=labels, include_lowest=True)
Train_trafic_2023['binned aankomst vertraging'] = Train_trafic_2023['binned aankomst vertraging'].astype(str)

counts = Train_trafic_2023['binned aankomst vertraging'].value_counts(sort=False)
fig = px.pie(values=counts, names=counts.index, title='aankomst vertraging in minuten.')
fig.show()

als ik naar wat de twee chards zeggen lijken ze goed evereen te komen. dit betekend dat meestal als een trein te laat binnen komt zal hij waarschijnlijk ook die zelfde verschil te laat vertrekken. en daarmee ga ik nu kijken wat de vertragin in minuten is in een tabel.

met deze table zal ik kijken of er op een bepaalde lijn meer vertragin is dan op een ander. hiervoor moet ik wel oppassen en kijken of ik niet te vaak de zelfde lijn meenemen. en ik pak de mean omdat ik de average vertaging van de ritten op een dag zien.

In [122]:

# Service:RDT-ID, Service:Date

grouped = Train_trafic_2023.groupby(['Service:Date', 'Service:RDT-ID'])['Stop:Arrival delay'].mean()

# To convert it back to a DataFrame
df_avarage_delay_day = grouped.reset_index(name='Stop:Arrival delay')

grouped = df_avarage_delay_day.groupby(['Service:Date'])['Stop:Arrival delay'].mean()

df_avarage_delay_day = grouped.reset_index(name='Stop:Arrival delay')

display(df_avarage_delay_day)

fig = px.bar(df_avarage_delay_day, x='Service:Date', y='Stop:Arrival delay')
fig.show()

,Service:Date,Stop:Arrival delay
0,2023-01-01,0.680368
1,2023-01-02,0.596654
2,2023-01-03,0.565669
3,2023-01-04,0.781114
4,2023-01-05,0.673781
...,...,...
85,2023-03-27,0.799672
86,2023-03-28,0.964126
87,2023-03-29,0.642580
88,2023-03-30,0.710096


in te grafiek hierboven is te zien dat het schommeeld door de maanden. dit kan belangerijk zijn voor het testen van omstandigheden.

### 🌥️ het weer
in het weer bestaan veel colomen en ik kijk even wat er nodig is en wat niet. daarvoor ga ik het gemidelde berekenene en kijken hoe ver ik daarmee van het antwoord. als het in de buurt zit kan ik zeggen dat de avarage goed is voor nederland als het niet zo is moet ik even kijken wat ik dan ga doen.


In [123]:
grouped = weather_2023.groupby(by=['start_time']).mean()

db_weather_single_day = grouped.reset_index()

db_weather_single_day = db_weather_single_day.drop(columns=['STN'])

display(db_weather_single_day.head(24))

df_first_date = weather_2023[weather_2023['start_time'] == weather_2023['start_time'].min()]

display(df_first_date[df_first_date['temp'].notna()].sort_values('temp'))


,start_time,DD,FH,FF,FX,temp,T10N,TD,SQ,Q,...,VV,N,U,WW,IX,M,R,S,O,Y
0,2023-01-01 00:00:00,215.555556,110.666667,110.000000,172.222222,140.617647,NaN,80.382353,0.00000,0.00000,...,73.28,8.00,67.558824,36.062500,6.152174,0.0,0.56,0.0,0.0,0.0
1,2023-01-01 01:00:00,215.333333,108.000000,106.444444,167.333333,138.029412,NaN,80.764706,0.00000,0.00000,...,74.28,8.00,68.941176,49.187500,6.152174,0.0,0.64,0.0,0.0,0.0
2,2023-01-01 02:00:00,220.222222,107.333333,106.888889,168.000000,135.529412,NaN,79.941176,0.00000,0.00000,...,74.12,8.00,69.764706,30.631579,6.282609,0.0,0.48,0.0,0.0,0.0
3,2023-01-01 03:00:00,223.111111,104.000000,103.333333,165.111111,131.323529,NaN,80.911765,0.00000,0.00000,...,73.20,7.95,72.147059,34.062500,6.152174,0.0,0.64,0.0,0.0,0.0
4,2023-01-01 04:00:00,224.444444,99.111111,99.111111,158.444444,125.970588,NaN,79.970588,0.00000,0.00000,...,73.88,8.00,73.911765,20.833333,5.978261,0.0,0.40,0.0,0.0,0.0
5,2023-01-01 05:00:00,224.666667,95.333333,94.222222,148.444444,122.558824,114.294118,76.294118,0.00000,0.00000,...,73.08,8.00,73.529412,11.791667,6.500000,0.0,0.20,0.0,0.0,0.0
6,2023-01-01 06:00:00,224.666667,91.333333,89.555556,140.222222,117.941176,NaN,75.352941,0.00000,0.00000,...,70.80,8.00,75.117647,45.666667,5.978261,0.0,0.40,0.0,0.0,0.0
7,2023-01-01 07:00:00,222.666667,87.111111,83.777778,134.444444,113.647059,NaN,75.970588,0.00000,0.25000,...,69.88,8.00,77.529412,36.153846,6.021739,0.0,0.40,0.0,0.0,0.0
8,2023-01-01 08:00:00,219.111111,81.777778,81.555556,127.555556,112.441176,NaN,78.558824,0.18750,5.62500,...,69.40,8.00,79.558824,22.785714,6.065217,0.0,0.24,0.0,0.0,0.0
9,2023-01-01 09:00:00,215.111111,76.666667,76.444444,119.555556,114.911765,NaN,81.676471,2.62500,25.12500,...,69.56,8.00,79.852941,28.166667,5.717391,0.0,0.16,0.0,0.0,0.0


,STN,start_time,DD,FH,FF,FX,temp,T10N,TD,SQ,...,VV,N,U,WW,IX,M,R,S,O,Y
17280,251,2023-01-01,220.0,140.0,130.0,190.0,108.0,NaN,88.0,0.0,...,58.0,NaN,87.0,10.0,7,0.0,0.0,0.0,0.0,0.0
6480,235,2023-01-01,220.0,120.0,120.0,180.0,112.0,NaN,95.0,0.0,...,56.0,8.0,89.0,52.0,7,0.0,1.0,0.0,0.0,0.0
10800,242,2023-01-01,220.0,160.0,150.0,210.0,114.0,NaN,91.0,NaN,...,57.0,8.0,86.0,10.0,7,0.0,0.0,0.0,0.0,0.0
25920,267,2023-01-01,220.0,130.0,130.0,200.0,121.0,NaN,89.0,0.0,...,64.0,8.0,81.0,57.0,7,0.0,1.0,0.0,0.0,0.0
19440,257,2023-01-01,NaN,NaN,NaN,NaN,123.0,NaN,83.0,0.0,...,NaN,NaN,76.0,NaN,6,NaN,NaN,NaN,NaN,NaN
49680,286,2023-01-01,210.0,110.0,100.0,170.0,128.0,NaN,98.0,0.0,...,NaN,NaN,82.0,NaN,6,NaN,NaN,NaN,NaN,NaN
36720,277,2023-01-01,220.0,120.0,130.0,200.0,130.0,NaN,92.0,0.0,...,NaN,NaN,78.0,NaN,6,NaN,NaN,NaN,NaN,NaN
32400,273,2023-01-01,220.0,80.0,70.0,130.0,131.0,NaN,93.0,0.0,...,64.0,NaN,77.0,23.0,7,0.0,1.0,0.0,0.0,0.0
56160,310,2023-01-01,200.0,140.0,140.0,180.0,131.0,NaN,79.0,0.0,...,75.0,8.0,70.0,NaN,5,0.0,0.0,0.0,0.0,0.0
30240,270,2023-01-01,230.0,140.0,140.0,230.0,132.0,NaN,94.0,0.0,...,74.0,8.0,78.0,23.0,7,0.0,1.0,0.0,0.0,0.0


het ziet er naar out dat met de temperatuur het ziet er uit dat er een ~4 grade verschil tussen de hoogste en laagste temperatuur. en wel 4 grade verschill niet veel is het kan er tot leiden dat mijn data niet meer klopt. dus het ziet er naar uit dat ik toch het verschill tussen de stations moet vinden en de dichtsbijzijnde weer station.

### 🏟️ trein station 

ik moet als eers erachter komen welke trein stations ik nodig heb voor mijn datasets. en dat is wat ik hieronder doe.

In [124]:
europa_train_station_locations = europa_train_station_locations[europa_train_station_locations['name'].isin(Train_trafic_2023['Stop:Station name'])]

display(europa_train_station_locations)

,name,name_norm,latitude,longitude,country
4495,amsterdam centraal,Amsterdam Centraal,52.379190,4.899426,NL
4504,antwerpen-centraal,Antwerpen-Centraal,51.215527,4.421005,BE
4529,liège-guillemins,Liege-Guillemins,50.624297,5.566667,BE
5098,aachen hbf,Aachen Hbf,50.767802,6.091495,DE
5100,aachen west,Aachen West,50.780360,6.070712,DE
...,...,...,...,...,...
21407,zevenaar,Zevenaar,51.923054,6.071944,NL
21408,zuidbroek,Zuidbroek,53.159612,6.867914,NL
21409,zuidhorn,Zuidhorn,53.248629,6.406236,NL
26794,utrecht vaartsche rijn,Utrecht Vaartsche Rijn,52.078611,5.122500,NL


met deze informatie kan ik mijn trein stations coördinaten geven zodat ik de dichtsbijzijnde weer station kan gebruiken.

maar om dat te testen doe ik hier eerst een test gebaseerd op 1 meetpunt moment.

hier gebrouk ik de BellTree funktie van de neighbor funkries van sklearn.neighbors. hiermee probeer ik de tempraturen van de stations te ontdekken.

In [125]:
from sklearn.neighbors import BallTree

europa_train_station_locations['latitude_rad'] = np.radians(europa_train_station_locations['latitude'])
europa_train_station_locations['longitude_rad'] = np.radians(europa_train_station_locations['longitude'])

weather_station_location['latitude_rad'] = np.radians(weather_station_location['LAT(north)'])
weather_station_location['longitude_rad'] = np.radians(weather_station_location['LON(east)'])

tree = BallTree(weather_station_location[['latitude_rad', 'longitude_rad']].dropna().values, metric='haversine')

indices = tree.query(europa_train_station_locations[['latitude_rad', 'longitude_rad']].dropna().values, return_distance=False, k=3)

display(indices)

display(europa_train_station_locations)

df_mergid_first_date_stations = df_first_date.merge(weather_station_location, on='STN', how='inner')

display(df_mergid_first_date_stations)

nearest_neighbors_weather_station_location = weather_station_location.iloc[indices.flatten()]

df_mergid_first_date_stations.reset_index(drop=True, inplace=True)
nearest_neighbors_weather_station_location.reset_index(drop=True, inplace=True)

combined_df = pd.concat([df_mergid_first_date_stations, nearest_neighbors_weather_station_location], axis=1)

display(combined_df.sample(sample_size))


array([[ 5, 10,  3],
       [39, 38, 34],
       [48, 47, 45],
       ...,
       [22, 19, 24],
       [12, 13, 42],
       [24, 22, 25]], dtype=int64)

,name,name_norm,latitude,longitude,country,latitude_rad,longitude_rad
4495,amsterdam centraal,Amsterdam Centraal,52.379190,4.899426,NL,0.914189,0.085511
4504,antwerpen-centraal,Antwerpen-Centraal,51.215527,4.421005,BE,0.893880,0.077161
4529,liège-guillemins,Liege-Guillemins,50.624297,5.566667,BE,0.883561,0.097157
5098,aachen hbf,Aachen Hbf,50.767802,6.091495,DE,0.886065,0.106317
5100,aachen west,Aachen West,50.780360,6.070712,DE,0.886284,0.105954
...,...,...,...,...,...,...,...
21407,zevenaar,Zevenaar,51.923054,6.071944,NL,0.906228,0.105975
21408,zuidbroek,Zuidbroek,53.159612,6.867914,NL,0.927810,0.119868
21409,zuidhorn,Zuidhorn,53.248629,6.406236,NL,0.929364,0.111810
26794,utrecht vaartsche rijn,Utrecht Vaartsche Rijn,52.078611,5.122500,NL,0.908943,0.089404


,STN,start_time,DD,FH,FF,FX,temp,T10N,TD,SQ,...,R,S,O,Y,LON(east),LAT(north),ALT(m),NAME,latitude_rad,longitude_rad
0,209,2023-01-01,220.0,140.0,150.0,170.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.518,52.465,0.0,IJmond,0.915687,0.078854
1,215,2023-01-01,220.0,110.0,130.0,200.0,142.0,NaN,84.0,0.0,...,0.0,0.0,0.0,0.0,4.437,52.141,-1.1,Voorschoten,0.910032,0.077440
2,225,2023-01-01,210.0,150.0,160.0,180.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.555,52.463,4.4,IJmuiden,0.915652,0.079500
3,235,2023-01-01,220.0,120.0,120.0,180.0,112.0,NaN,95.0,0.0,...,1.0,0.0,0.0,0.0,4.781,52.928,1.2,DeKooy,0.923768,0.083444
4,240,2023-01-01,220.0,110.0,110.0,150.0,138.0,NaN,92.0,0.0,...,1.0,0.0,0.0,0.0,4.790,52.318,-3.3,Schiphol,0.913121,0.083601
5,242,2023-01-01,220.0,160.0,150.0,210.0,114.0,NaN,91.0,NaN,...,0.0,0.0,0.0,0.0,4.921,53.241,10.8,Vlieland,0.929231,0.085888
6,248,2023-01-01,230.0,120.0,110.0,210.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.174,52.634,0.8,Wijdenes,0.918637,0.090303
7,249,2023-01-01,230.0,140.0,130.0,230.0,136.0,NaN,98.0,0.0,...,1.0,0.0,0.0,0.0,4.979,52.644,-2.4,Berkhout,0.918811,0.086900
8,251,2023-01-01,220.0,140.0,130.0,190.0,108.0,NaN,88.0,0.0,...,0.0,0.0,0.0,0.0,5.346,53.392,0.7,HoornTerschelling,0.931866,0.093305
9,257,2023-01-01,NaN,NaN,NaN,NaN,123.0,NaN,83.0,0.0,...,NaN,NaN,NaN,NaN,4.603,52.506,8.5,WijkaanZee,0.916403,0.080338


,STN,start_time,DD,FH,FF,FX,temp,T10N,TD,SQ,...,NAME,latitude_rad,longitude_rad,STN,LON(east),LAT(north),ALT(m),NAME,latitude_rad,longitude_rad
229,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,280,6.585,53.125,5.2,Eelde,0.927206,0.114930
335,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,278,6.259,52.435,3.6,Heino,0.915163,0.109240
1405,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,290,6.891,52.274,34.8,Twenthe,0.912353,0.120271
1031,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,356,5.146,51.859,0.7,Herwijnen,0.905110,0.089815
878,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,348,4.926,51.970,-0.7,CabauwMast,0.907048,0.085975
498,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,340,4.342,51.449,19.2,Woensdrecht,0.897954,0.075782
315,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,290,6.891,52.274,34.8,Twenthe,0.912353,0.120271
317,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,278,6.259,52.435,3.6,Heino,0.915163,0.109240
1516,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,280,6.585,53.125,5.2,Eelde,0.927206,0.114930
510,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,340,4.342,51.449,19.2,Woensdrecht,0.897954,0.075782
